# Travelling Salesman Problem

The Traveling Salesman Problem (TSP) is a classic optimization problem in the field of operations research and computer science. Given a list of cities and the distances between each pair of cities, the task is to find the shortest possible route that visits each city exactly once and returns to the starting city. The TSP is an NP-hard problem, meaning that there is no known algorithm that can solve it efficiently for large instances in polynomial time. However, there are several approaches to tackle the problem, ranging from exact algorithms to heuristic and approximation methods.

1. Exact algorithms: These algorithms aim to find the optimal solution for small to medium-sized instances of the TSP. The most well-known exact algorithm is the Held-Karp algorithm, also known as the dynamic programming algorithm. It has a time complexity of O(n^2 * 2^n) and is not practical for large instances due to its exponential growth.

2. Heuristic algorithms: Heuristic algorithms are methods that provide a reasonably good solution in a more efficient time frame compared to exact algorithms. Examples include the Nearest Neighbor algorithm, the Insertion algorithm, and the Genetic Algorithm.

3. Approximation algorithms: These algorithms guarantee a solution that is within a certain factor of the optimal solution. The most famous approximation algorithm for TSP is the Christofides algorithm, which guarantees a solution that is at most 3/2 times the length of the optimal solution.

4. Metaheuristic algorithms: These are high-level strategies that guide other algorithms to explore the solution space efficiently. Examples include Simulated Annealing, Ant Colony Optimization, and Particle Swarm Optimization.

It's essential to choose the appropriate algorithm based on the size of the TSP instance and the desired level of optimality. For small instances, exact algorithms can be used, but for larger instances, heuristic or approximation algorithms are more practical.

The TSP has numerous real-world applications, such as planning vehicle routes for delivery services, optimizing circuit board manufacturing, and DNA sequencing. Due to its computational complexity and practical significance, the TSP remains a widely studied problem in the field of optimization and computer science.



## Our Approach:
We will modify the Stochastic Search algorithm to solve the TSP optimisation problem. The initial population generate children by random swapping the elements in the parent solution. The total distance covered is the objective function for this problem which we want to minimize. We will start from city '0' and come back to city '0' while covering all the cities.

### Import necessary Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### Read the Distance Matrix

In [2]:
dist_matr = np.genfromtxt("/content/dist_matrix_TSP.csv",delimiter=',')
print(dist_matr)

OSError: /content/dist_matrix_TSP.csv not found.

In [ ]:
print(dist_matr.shape)

In [ ]:
print((dist_matr==dist_matr.T).all())

### Define the Distance function

In [ ]:
def dist_func(a,dist_matrix):  ## Function to calculate the distance covered for a vector solution
    total_distance = 0
    i = 0
    for j in range(len(a)):
        total_distance += dist_matrix[a[j]][i]
        i = a[j]
    total_distance += dist_matrix[i][0]
    return total_distance

### Function to Swap two elements

In [ ]:
def swap(x): # Helper function to generate children from the parent solution by swapping two elements
    sampled_idx = np.random.choice(len(x),2,replace=False)
    x[sampled_idx[0]],x[sampled_idx[1]] = x[sampled_idx[1]],x[sampled_idx[0]]
    return x

### Random re-initialize points

In [ ]:
def rrI(arr,rri_num):  # Function to generate the Random Reinitialisation vectors
    rri = []
    for i in range(rri_num):
        new = np.random.permutation(arr)
        rri.append(new)
    rri = np.matrix(rri)
    return rri

### Function to obtain Optimum Path

In [ ]:
def Optimum_path(path,dist_matrix,popsize,rri_num,maxIter):
    ## Generate the initial population
    population = []
    for i in range(popsize):
        random_path = np.random.permutation(path)
        population.append(random_path)
    population = np.matrix(population)
    # print(population)
    ## Main Loop begins
    for itr in range(maxIter):
        y_arr = []
        for i in range(population.shape[0]):
            a = np.ravel(population[i,:])
            y = dist_func(a,dist_matrix)
            y_arr.append(y)

        u_arr = [y_arr[i]-min(y_arr) for i in range(len(y_arr))]
        fit_score = [u_arr[i]/sum(u_arr) for i in range(len(u_arr))]
        # print('fit scores', fit_score)
        child_num = [int(fit_score[i]*popsize) for i in range(len(y_arr))]
        # print(child_num)
        child_arr = []
    ## Generate the children population
        for i in range(len(y_arr)):
            x_i = np.ravel(population[i,:])
            for j in range(child_num[i]):
                c = swap(x_i)
                child_arr.append(c)
        child_mat = np.matrix(child_arr)
    ## Generate the RRI population
        rri = rrI(path,rri_num)

        total_pop = np.vstack((population,child_mat,rri))
        # print(population.shape,child_mat.shape,rri.shape, total_pop.shape)
        fitness_arr = []

        for i in range(total_pop.shape[0]):
            a = np.ravel(total_pop[i,:])
            d = dist_func(a,dist_matrix)
            fitness_arr.append(d)

        fitness_arr = np.array(fitness_arr)
        # print(fitness_arr)
        min_idx = np.argsort(fitness_arr)
        # print(min_idx)
        useful_idx = min_idx[:popsize]
        useful_dist = fitness_arr[useful_idx]

        min_dist_idx = useful_idx[0]
        best_memb = total_pop[min_dist_idx,:]
        best_dist = useful_dist[0]

        print("Iteration: ",itr+1)
        print("Best Path till now is {}".format(best_memb))
        print("Best Distance till now is {}".format(best_dist))
        popList = []
        for i in range(len(useful_idx)):
            idx = useful_idx[i]
            popMember = np.ravel(total_pop[idx,:])
            popList.append(popMember)
        population = np.matrix(popList)
    return best_memb

In [ ]:
path = np.arange(1,26,1)
print(path)

In [ ]:
opt_path = Optimum_path(path,dist_matr,300,200,3000)

## Optimal Answer to this problem

In [ ]:
ans = np.array([24,23,22,25,21,20,16,17,19,18,15,10,11,12,14,13,9,8,7,6,4,5,3,2,1])
print("The minimum distance path is:")
print(str(0)+"->",end="")
for i in range(len(ans)):
    print(str(ans[i])+"->",end="")
print(str(0))
print("Minimum possible distance is {}".format(dist_func(ans,dist_matr)))

In [ ]:
from sklearn.manifold import MDS #Multi-dimensional scaling
model = MDS(n_components=2,dissimilarity='precomputed',random_state=1)
out = model.fit_transform(dist_matr)

n = np.arange(26)
x = out[:,0]
y = out[:,1]
fig,ax = plt.subplots(figsize=(10,8))
ax.scatter(x,y)
ax.arrow(x[0],y[0],x[ans[0]]-x[0],y[ans[0]]-y[0])
for i,txt in enumerate(n):
    if(i<24):
        ax.arrow(x[ans[i]],y[ans[i]],x[ans[i+1]]-x[ans[i]],y[ans[i+1]]-y[ans[i]])
    ax.annotate(txt,(x[i],y[i]))
plt.title("TSP Solution")
plt.show()